In [1]:
%env SPECTRAL_CONNECTIVITY_ENABLE_GPU=true

env: SPECTRAL_CONNECTIVITY_ENABLE_GPU=true


In [2]:
import os
os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')


import cupy as xp
from cupyx.scipy.fft import ifft
from cupyx.scipy.sparse.linalg import svds
from spectral_connectivity import Multitaper, Connectivity
import importlib
import pandas as pd
from itertools import combinations
import os
from bidict import bidict
import lfp.lfp_analysis.LFP_collection as LFP_collection
import pickle 
def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl
    Returns:
        none
    """
    with open(file_name, "wb") as file:
        pickle.dump(thing_to_pickle, file)
def unpickle_this(pickle_file):
    """
    Unpickles things
    Args (1):
        file_name: str, pickle filename that already exists and ends with .pkl
    Returns:
        pickled item
    """
    with open(pickle_file, "rb") as file:
        return pickle.load(file)

df = pd.read_excel(r"lfp/channel_mapping_sme.xlsx")
spike_cols = [col for col in df.columns if 'spike_interface_' in col.lower()]

# Extract brain regions from column names
# Assumes format 'spike_interface_REGION'
brain_regions = [col.split('spike_interface_')[1] for col in spike_cols]

# Create nested dictionary
subject_to_channel_dict = {}







for _, row in df.iterrows():
    subject = row['Subject'].astype(str)
    # Initialize inner dictionary for this subject
    subject_to_channel_dict[subject] = {}
    
    # Populate inner dictionary with brain region: spike value pairs
    for col, region in zip(spike_cols, brain_regions):
        subject_to_channel_dict[subject][region] = int(row[col])
behavior_dicts = {}
#pickle_this(subject_to_channel_dict)

def make_recording_to_subj_dict(data_path):
    recording_to_subject = {}
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith('merged.rec'):
                subject = str(int((file.split("_")[0]))/10)
                recording_to_subject[file] = subject
                behavior_dicts[file] = {}
    return recording_to_subject

# def process(data_path):
#     recording_to_subject = make_recording_to_subj_dict(data_path)
#     print(recording_to_subject)
#     collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 4)
#     #collection.process()
#     return collection    
    

    



In [3]:
import lfp.lfp_analysis.LFP_recording as rec 
rec_path = r"data/new_object_control_data/20250618_113931_object_control_subj_1-2_and_6-1.rec/20250618_113931_object_control_subj_1-2_merged.rec"
rec_object = rec.LFPRecording(subject = '1.2', channel_dict = {'mPFC': 2,
                                 'NAc': 28,
                                 'MD': 29,
                                 'LH': 30,
                                 'BLA': 31}, merged_rec_path = rec_path)



Processing 20250618_113931_object_control_subj_1-2_merged.rec
Found first timestamp


In [4]:
trodes_rec = rec_object._read_trodes()
traces = trodes_rec.get_traces()

Processing 20250618_113931_object_control_subj_1-2_merged.rec


In [5]:
print(traces.shape)
print(trodes_rec)


(0, 32)
ResampleRecording: 32 channels - 1 segments - 1.0kHz - 1939.501s


In [6]:
traces = trodes_rec.get_traces(['2','28','29','30','31'], start_frame = None)

TypeError: list indices must be integers or slices, not list

In [26]:
rec_object.traces.shape

(0, 5)

In [3]:
data_path = r"data/new_object_control_data"
recording_to_subject = make_recording_to_subj_dict(data_path)


In [4]:
collection = LFP_collection.LFPCollection(subject_to_channel_dict, data_path, recording_to_subject, 5)


Processing 22_object_merged.rec
Found first timestamp
Processing 44_object_merged.rec
Found first timestamp
Processing 32_object_merged.rec
Found first timestamp
Processing 31_object_merged.rec
Found first timestamp
Processing 23_object_merged.rec
Found first timestamp
Processing 41_object_merged.rec
Found first timestamp


In [5]:
collection.process()

  0%|          | 0/6 [00:00<?, ?it/s]

processing 22_object_merged.rec
Filename: /blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys/lfp/lfp_analysis/preprocessor.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    13    443.3 MiB    443.3 MiB           1   @profile
    14                                         def preprocess(traces, threshold, scaling):
    15                                             # brain_region_dict, traces = map_to_region(all_traces, subject_region_dict)
    16    512.3 MiB     69.1 MiB           1       voltage_scaled_traces = scale_voltage(traces, scaling)
    17    581.4 MiB     69.1 MiB           1       zscored_traces = zscore(voltage_scaled_traces)
    18    650.3 MiB     68.9 MiB           1       filtered_traces = filter(zscored_traces, voltage_scaled_traces, threshold)
    19    710.9 MiB     60.5 MiB           1       rms_traces = root_mean_square(filtered_traces)
    20    710.9 MiB      0.0 MiB           1       return rms_traces


RMS Traces calcul

Maximum iterations reached. 3335 of 3620 converged
Maximum iterations reached. 3424 of 3620 converged
Maximum iterations reached. 3314 of 3620 converged
Maximum iterations reached. 3319 of 3620 converged
Maximum iterations reached. 3330 of 3620 converged
Maximum iterations reached. 3271 of 3620 converged
Maximum iterations reached. 3226 of 3620 converged
Maximum iterations reached. 3350 of 3620 converged
Maximum iterations reached. 3369 of 3620 converged
Maximum iterations reached. 3250 of 3620 converged
 17%|█▋        | 1/6 [00:43<03:38, 43.78s/it]

Granger's causality calculated
processing 44_object_merged.rec
Filename: /blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys/lfp/lfp_analysis/preprocessor.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    13   1693.6 MiB   1693.6 MiB           1   @profile
    14                                         def preprocess(traces, threshold, scaling):
    15                                             # brain_region_dict, traces = map_to_region(all_traces, subject_region_dict)
    16   1762.6 MiB     68.9 MiB           1       voltage_scaled_traces = scale_voltage(traces, scaling)
    17   1831.7 MiB     69.1 MiB           1       zscored_traces = zscore(voltage_scaled_traces)
    18   1909.0 MiB     77.3 MiB           1       filtered_traces = filter(zscored_traces, voltage_scaled_traces, threshold)
    19   1969.4 MiB     60.4 MiB           1       rms_traces = root_mean_square(filtered_traces)
    20   1969.4 MiB      0.0 MiB           1       return

Maximum iterations reached. 3529 of 3618 converged
Maximum iterations reached. 3560 of 3618 converged
Maximum iterations reached. 3559 of 3618 converged
Maximum iterations reached. 3543 of 3618 converged
Maximum iterations reached. 3531 of 3618 converged
Maximum iterations reached. 3549 of 3618 converged
Maximum iterations reached. 3533 of 3618 converged
Maximum iterations reached. 3582 of 3618 converged
Maximum iterations reached. 3579 of 3618 converged
Maximum iterations reached. 3570 of 3618 converged
 33%|███▎      | 2/6 [01:08<02:10, 32.68s/it]

...

Granger's causality calculated
processing 23_object_merged.rec
Filename: /blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys/lfp/lfp_analysis/preprocessor.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    13   4197.3 MiB   4197.3 MiB           1   @profile
    14                                         def preprocess(traces, threshold, scaling):
    15                                             # brain_region_dict, traces = map_to_region(all_traces, subject_region_dict)
    16   4267.0 MiB     69.8 MiB           1       voltage_scaled_traces = scale_voltage(traces, scaling)
    17   4337.0 MiB     70.0 MiB           1       zscored_traces = zscore(voltage_scaled_traces)
    18   4415.3 MiB     78.3 MiB           1       filtered_traces = filter(zscored_traces, voltage_scaled_traces, threshold)
    19   4476.6 MiB     61.2 MiB           1       rms_traces = root_mean_square(filtered_traces)
    20   4476.6 MiB      0.0 MiB           1       return

Maximum iterations reached. 3632 of 3660 converged
Maximum iterations reached. 3640 of 3660 converged
Maximum iterations reached. 3632 of 3660 converged
Maximum iterations reached. 3582 of 3660 converged
Maximum iterations reached. 3630 of 3660 converged
Maximum iterations reached. 3636 of 3660 converged
Maximum iterations reached. 3586 of 3660 converged
Maximum iterations reached. 3642 of 3660 converged
Maximum iterations reached. 3602 of 3660 converged
Maximum iterations reached. 3589 of 3660 converged
 83%|████████▎ | 5/6 [02:24<00:26, 26.75s/it]

Granger's causality calculated
processing 41_object_merged.rec
Filename: /blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys/lfp/lfp_analysis/preprocessor.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    13   5035.2 MiB   5035.2 MiB           1   @profile
    14                                         def preprocess(traces, threshold, scaling):
    15                                             # brain_region_dict, traces = map_to_region(all_traces, subject_region_dict)
    16   5104.9 MiB     69.7 MiB           1       voltage_scaled_traces = scale_voltage(traces, scaling)
    17   5174.9 MiB     69.9 MiB           1       zscored_traces = zscore(voltage_scaled_traces)
    18   5253.1 MiB     78.3 MiB           1       filtered_traces = filter(zscored_traces, voltage_scaled_traces, threshold)
    19   5314.4 MiB     61.3 MiB           1       rms_traces = root_mean_square(filtered_traces)
    20   5314.4 MiB      0.0 MiB           1       return

Maximum iterations reached. 3608 of 3659 converged
Maximum iterations reached. 3628 of 3659 converged
Maximum iterations reached. 3618 of 3659 converged
Maximum iterations reached. 2865 of 3659 converged
Maximum iterations reached. 3605 of 3659 converged
Maximum iterations reached. 3602 of 3659 converged
Maximum iterations reached. 2854 of 3659 converged
Maximum iterations reached. 3637 of 3659 converged
Maximum iterations reached. 2879 of 3659 converged
Maximum iterations reached. 2870 of 3659 converged
100%|██████████| 6/6 [02:49<00:00, 28.27s/it]

Granger's causality calculated


In [6]:
LFP_collection.LFPCollection.save_to_json(collection, r"data/object_control_sd5")